# Sentiment Analysis using Doc2Vec

Dataset: Cornell IMDB movie review corpus (http://www.cs.cornell.edu/people/pabo/movie-review-data/).

Original source code used in this demo can be found at https://github.com/linanqiu/word2vec-sentiments

## Setup

### Modules

We use `gensim`, since `gensim` has a much more readable implementation of Word2Vec (and Doc2Vec). Bless those guys. We also use `numpy` for general array manipulation, and `sklearn` for Logistic Regression classifier.

In [21]:
import gensim
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

import numpy

from sklearn.linear_model import LogisticRegression

import random

### Input Format

We can't input the raw reviews from the Cornell movie review data repository. Instead, we clean them up by converting everything to lower case and removing punctuation. I did this via bash, and you can do this easily via Python, JS, or your favorite poison. This step is trivial.

The result is to have five documents:

- `test-neg.txt`: 12500 negative movie reviews from the test data
- `test-pos.txt`: 12500 positive movie reviews from the test data
- `train-neg.txt`: 12500 negative movie reviews from the training data
- `train-pos.txt`: 12500 positive movie reviews from the training data
- `train-unsup.txt`: 50000 Unlabelled movie reviews

Each of the reviews should be formatted as such:

```
once again mr costner has dragged out a movie for far longer than necessary aside from the terrific sea rescue sequences of which there are very few i just did not care about any of the characters most of us have ghosts in the closet and costner s character are realized early on and then forgotten until much later by which time i did not care the character we should really care about is a very cocky overconfident ashton kutcher the problem is he comes off as kid who thinks he s better than anyone else around him and shows no signs of a cluttered closet his only obstacle appears to be winning over costner finally when we are well past the half way point of this stinker costner tells us all about kutcher s ghosts we are told why kutcher is driven to be the best with no prior inkling or foreshadowing no magic here it was all i could do to keep from turning it off an hour in
this is an example of why the majority of action films are the same generic and boring there s really nothing worth watching here a complete waste of the then barely tapped talents of ice t and ice cube who ve each proven many times over that they are capable of acting and acting well don t bother with this one go see new jack city ricochet or watch new york undercover for ice t or boyz n the hood higher learning or friday for ice cube and see the real deal ice t s horribly cliched dialogue alone makes this film grate at the teeth and i m still wondering what the heck bill paxton was doing in this film and why the heck does he always play the exact same character from aliens onward every film i ve seen with bill paxton has him playing the exact same irritating character and at least in aliens his character died which made it somewhat gratifying overall this is second rate action trash there are countless better films to see and if you really want to see this one watch judgement night which is practically a carbon copy but has better acting and a better script the only thing that made this at all worth watching was a decent hand on the camera the cinematography was almost refreshing which comes close to making up for the horrible film itself but not quite
```

The sample up there contains two movie reviews, each one taking up one entire line. Yes, **each document should be on one line, separated by new lines**. This is extremely important, because our parser depends on this to identify sentences.

### Feeding Data to Doc2Vec
                    

In [22]:
import smart_open

sources = {'data/test-neg.txt':'TEST_NEG', 'data/test-pos.txt':'TEST_POS', 'data/train-neg.txt':'TRAIN_NEG', 'data/train-pos.txt':'TRAIN_POS', 'data/train-unsup.txt':'TRAIN_UNS'}

taggedDocs = []
for source, prefix in sources.items():
    with smart_open.smart_open(source, encoding="iso-8859-1") as fin:
        for item_no, line in enumerate(fin):
            tagd = TaggedDocument(gensim.utils.simple_preprocess(line), [prefix + '_%s' % item_no])
            taggedDocs.append(tagd)

## Model

### Building the Vocabulary Table

Doc2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them). So we feed it the array of sentences. `model.build_vocab` takes an array of tagged documents.

If you're curious about the parameters, do read the Word2Vec documentation. Otherwise, here's a quick rundown:

- `min_count`: ignore all words with total frequency lower than this. You have to set this to 1, since the sentence labels only appear once. Setting it any higher than 1 will miss out on the sentences.
- `window`: the maximum distance between the current and predicted word within a sentence. Word2Vec uses a skip-gram model, and this is simply the window size of the skip-gram model.
- `size`: dimensionality of the feature vectors in output. 100 is a good number. If you're extreme, you can go up to around 400.
- `sample`: threshold for configuring which higher-frequency words are randomly downsampled
- `workers`: use this many worker threads to train the model 

In [23]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=2)

model.build_vocab(taggedDocs)

### Training Doc2Vec

Now we train the model. The model is better trained if **in each training epoch, the sequence of sentences fed to the model is randomized**. 

In [24]:
for i in range(2):
    print("Iteration {}...".format(i))
    model.train(taggedDocs,total_examples=model.corpus_count, epochs=model.iter)

Iteration 0...
Iteration 1...


### Inspecting the Model

Let's see what our model gives. It seems that it has kind of understood the word `good`, since the most similar words to good are `glamorous`, `spectacular`, `astounding` etc. This is really awesome (and important), since we are doing sentiment analysis.

In [25]:
model.most_similar('good')

[('great', 0.8007386922836304),
 ('bad', 0.7223443388938904),
 ('decent', 0.7146170139312744),
 ('nice', 0.6722875833511353),
 ('alright', 0.658721923828125),
 ('really', 0.6445150375366211),
 ('fine', 0.6387638449668884),
 ('ok', 0.6241072416305542),
 ('solid', 0.6185141801834106),
 ('excellent', 0.6180463433265686)]

We can also prop the hood open and see what the model actually contains. This is each of the vectors of the words and sentences in the model. We can access all of them using `model.syn0` (for the geekier ones among you, `syn0` is simply the output layer of the shallow neural network). However, we don't want to use the entire `syn0` since that contains the vectors for the words as well, but we are only interested in the ones for sentences.

Here's a sample vector for the first sentence in the training set for negative reviews:

In [26]:
model.docvecs['TRAIN_NEG_0']

array([ 0.03947123,  0.05880485, -0.01933228,  0.00114341,  0.10921661,
       -0.0154533 , -0.13605359, -0.23536907,  0.12516667, -0.06158252,
       -0.06861397, -0.14788494,  0.10207406, -0.02832308, -0.08838165,
       -0.15984435, -0.06263673, -0.25901139,  0.14259511,  0.00256087,
       -0.02021833, -0.06175816, -0.36156526,  0.08720211,  0.05684499,
        0.06779034, -0.27391869, -0.01824627,  0.01464311,  0.31672773,
        0.19414639,  0.02110008, -0.17429337,  0.13508765, -0.02746822,
        0.11820269,  0.08111748, -0.28518471,  0.03762215, -0.0622735 ,
       -0.06172793,  0.33397967,  0.11939584, -0.13362409, -0.0908304 ,
        0.04566756,  0.24363555,  0.16273785,  0.19619484,  0.03780841,
        0.07736275, -0.1767472 ,  0.13852711, -0.04761354, -0.2635586 ,
        0.19032872,  0.28821182, -0.07301085, -0.09486395, -0.05014978,
        0.06874591,  0.01901783,  0.17222384,  0.07533717, -0.04251823,
        0.01952993, -0.13895859, -0.02806682,  0.05565381, -0.07

### Saving and Loading Models

To avoid training the model again, we can save it.

In [27]:
# model.save('data/imdb.d2v')

And load it.

In [28]:
# model = Doc2Vec.load('data/imdb.d2v')

## Classifying Sentiments

### Training Vectors

Now let's use these vectors to train a classifier. First, we must extract the training vectors. Remember that we have a total of 25000 training reviews, with equal numbers of positive and negative ones (12500 positive, 12500 negative).

Hence, we create a `numpy` array (since the classifier we use only takes numpy arrays. There are two parallel arrays, one containing the vectors (`train_arrays`) and the other containing the labels (`train_labels`).

We simply put the positive ones at the first half of the array, and the negative ones at the second half.

In [29]:
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model.docvecs[prefix_train_pos]
    train_arrays[12500 + i] = model.docvecs[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

The training array looks like this: rows and rows of vectors representing each sentence.

In [30]:
train_arrays

array([[-0.09566638,  0.21176524,  0.19502051, ..., -0.09677574,
         0.03370467,  0.00707885],
       [ 0.07708465,  0.76514679,  0.21094929, ..., -0.31213251,
        -0.19084196, -0.25557962],
       [-0.016037  ,  0.32469413,  0.09334482, ...,  0.01904216,
         0.17723377,  0.09686495],
       ..., 
       [-0.3370229 , -0.24464233, -0.08707684, ..., -0.0966578 ,
        -0.27211857,  0.26228032],
       [-0.18824543,  0.0037224 ,  0.239705  , ...,  0.08876734,
        -0.09274796,  0.12405054],
       [-0.04972176,  0.14120647,  0.25808707, ..., -0.11219119,
        -0.12986887,  0.01133876]])

The labels are simply category labels for the sentence vectors -- 1 representing positive and 0 for negative.

In [31]:
train_labels

array([ 1.,  1.,  1., ...,  0.,  0.,  0.])

### Testing Vectors

We do the same for testing data -- data that we are going to feed to the classifier after we've trained it using the training data. This allows us to evaluate our results. The process is pretty much the same as extracting the results for the training data.

In [32]:
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model.docvecs[prefix_test_pos]
    test_arrays[12500 + i] = model.docvecs[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

### Classification

Now we train a logistic regression classifier using the training data.

In [33]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

C:\Users\kulkarni\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

And find that we have achieved near 87% accuracy for sentiment analysis. This is rather incredible, given that we are only using a linear SVM and a very shallow neural network.

In [34]:
classifier.score(test_arrays, test_labels)

0.59923999999999999

## References

- Doc2vec: https://radimrehurek.com/gensim/models/doc2vec.html
- Paper that inspired this: http://arxiv.org/abs/1405.4053